In [9]:
import pandas as pd
import altair as alt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
from bs4 import BeautifulSoup
import requests

In [10]:
humanist = ["https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/", "https://humanist.kdl.kcl.ac.uk/Archives/Current/"]

In [31]:
volumes = []
for url in humanist:
    print(f"{url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all('a')
    for link in links:
        if link['href'].endswith('.txt'):
            print(f"Getting volume from {url + link['href']}")
            page_soup = BeautifulSoup(requests.get(url + link['href']).text, "html.parser")
            text = page_soup.get_text()
            volume_link = url + link['href']
            dates = link['href'].split('.')[1]
            data_dict = {'volume_text': text, 'volume_link': volume_link, 'volume_dates': dates}
            volumes.append(data_dict)
            

https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/
https://humanist.kdl.kcl.ac.uk/Archives/Current/


In [29]:
humanist_df = pd.read_csv("web_scraped_humanist_listserv.csv")

In [33]:
vectorizer = TfidfVectorizer(max_df=.7, min_df=1)

dates = humanist_df.inferred_start_year.tolist()
both_sections = []
for s in [volumes]:
    documents = s.volume_text.tolist()
    tfidf_results = []
    for counter, doc in enumerate(vectorizer.fit_transform(documents).toarray()
    ):
        tf_idf_tuples = list(zip(vectorizer.get_feature_names_out(), doc))
        one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
        one_doc_as_df['inferred_start_year'] = dates[counter]
        tfidf_results.append(one_doc_as_df)
        p = pd.concat(tfidf_results).sort_values('score',ascending=False).head(5)
    both_sections.append(p)

AttributeError: 'DataFrame' object has no attribute 'inferred_start_year'